# Automatizálás és processzek

Ha automatizálni szeretnénk folyamatokat (például elindítani, leállítani valamilyen szoftvert a gépen) elkerülhetetlen, hogy a python keretein is túllépjünk, sőt néha párhuzamosan kell dolgoznunk.

Nézzük meg hogyan tudnánk pythonból más programokat elindítani vagy (akár python) programokat egyszerre futtatni.

## Külső programok

Korábban már említettem, hogy a colab egy linux containert futtat, itt tehát linux parancsokat tudunk elindítani, de az alábbiak minden operációs rendszeren működnek, csak a programok nevét, esetleg elérési útját kell értelemszerűen módosítani.

Ha ilyesmivel szeretnénk foglalkozni, a legjobb barátunk a `subprocess` csomag!

In [ ]:
import subprocess

eredmeny = subprocess.run(
    ["ls", "-l"],       # vagy Windows-on: ["cmd", "/c", "dir"]
    capture_output=True, # a kimenetet is őrizd meg
    text=True,
    check=False
)

print("Return code:", eredmeny.returncode)
print("STDOUT:")
print(eredmeny.stdout) # szabványos kimenet
print("STDERR:")
print(eredmeny.stderr) # szabványos hiba kimenet


Return code: 0
STDOUT:
total 4
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data

STDERR:



Látható, hogy nem túl nehéz programokat futtatni (az első paraméter után minden más opcionális volt), egyszerűen csak egy listában át kell adnunk a futtatandó parancs részeit. Ha nem akarunk neki paramétereket átadni, akkor csak a nevét.

A visszatérési értéke viszont talán némi magyarázatot igényel:


**Return code (visszatérési kód)**: Minden program a futása végén ad egy egész számot, ami jelzi, hogy sikeresen lefutott-e.  
  * 0 → siker
  * nem 0 → valamilyen hiba vagy rendellenesség történt

**Szabványos kimenet (STDOUT)**: Ezek a futó program normál üzenetei, pl. amit "kiír a képernyőre". Ez az, amit a print() is használ Pythonban.

**Szabványos hiba (STDERR)**: A program hibajelzései, figyelmeztetései külön csatornán érkeznek. Ez akkor is külön van, ha a program egyébként jól fut.


## Több program futtatása egymás után



In [ ]:
import subprocess

parancsok = [
    ["python", "--version"],
    ["echo", "Hello"],
]

for cmd in parancsok:
    eredmeny = subprocess.run(cmd, capture_output=True, text=True)
    print(f"Parancs: {cmd}", end=' ')
    print("RC:", eredmeny.returncode, end=' ')
    print("OUT:", eredmeny.stdout.strip())

Parancs: ['python', '--version'] RC: 0 OUT: Python 3.12.12
Parancs: ['echo', 'Hello'] RC: 0 OUT: Hello


## Több program, de párhuzamosan
Ha nem szeretnéd megvárni a programok futását (például mert sokáig tart), elindíthatod őket "egyszerre" is.  (Figyelj, ez a kódblokk "sokáig" fut)


In [ ]:
import subprocess

parancsok = [
    ["sleep", "5"], # ez a parancs nem csinál semmit 5 másodpercig
    ["sleep", "7"],
    ["sleep", "3"],
]

folyamatok = []

# elindítjuk őket
for cmd in parancsok:
    p = subprocess.Popen(cmd)
    folyamatok.append(p)

# itt csinálhatunk valami mást...
# ...
# ...

# megvárjuk az összeset
for p in folyamatok:
    p.communicate()  # blokkol, amíg az adott folyamat be nem fejeződik



In [ ]:
!ps xa | grep 'python|sleep|grep'

## Több Python processz párhuzamos számításhoz

Már említettük, hogy a python egy processzormagon fut. A modern 8, 16 vagy még több magos processzorok világában ez némiképp pazarlásnak tűnhet, főleg, ha egy amúgy jól párhuzamosítható feladaton dolgozunk. (Mondjuk különálló, egymástól független fájlokat kell feldolgoznunk).

A multiprocessing csomag ebben segít minket. Itt elindíthatunk egy nagy adag számítást és megvárhatjuk míg mindegyik végez!

Minden számítás más paraméteren dolgozik (párhuzamosan)! Az alábbi kódban ezek a bemenetek egyszerű számok, de lehetne bármi más is.

In [ ]:
from multiprocessing import Pool
import math

def nehez_szamitas(x: int) -> float:
    # valami CPU-igényesebb dolog
    s = 0.0
    for i in range(100_000):
        s += math.sqrt(x * i + 1)
    return s

if __name__ == "__main__":  # Windows-on ezt KÖTELEZŐ használni
    bemenetek = [1, 2, 3, 4, 5, 6, 7, 8]

    with Pool(processes=4) as pool:  # 4 párhuzamos processz
        eredmenyek = pool.map(nehez_szamitas, bemenetek)

    print(eredmenyek)


Jó, hát sokáig azért gondolom nem kellett várnod 😀 Ezek a mai gépek pillanatok alatt végeznek el pár százezer gyökvonást. Na de azért vannak feladatok, amelyek már rajtuk is kifognak. Most már el tudod indítani a számításaidat párhuzamosan, egyszerre több magon!




## Több processz kézzel

Az előző példában a with context manager szuper módon leegyszerűsítette a dolgunkat. Néha előfordulhat, hogy kézzel szeretnéd elindítgatni és leállítani a folyamatokat. Ez sem különösebben nehéz, de azért kicsit többet kell gépelni:


In [ ]:
from multiprocessing import Process
import time # ez nem kell csak az alváshoz

def worker(nev: str, n: int) -> None:
  # itt mindenféle bonyolult dolgot csinálhatnánk....
  print(f"{nev} indul")
  time.sleep(n) # de most csak alszunk picit.
  print(f"{nev} kész")

if __name__ == "__main__":
  # létrehozunk két folyamatot
  p1 = Process(target=worker, args=("A folyamat", 2))
  p2 = Process(target=worker, args=("B folyamat", 3))

  # elindítjuk őket...
  p1.start()
  p2.start()

  # megvárjuk mindkettőt
  p1.join()
  p2.join()

print("Minden folyamat befejeződött.")


## Mikor melyiket használd:

**subprocess** akkor hasznos, ha:
- nem Python-programot akarsz futtatni (pl. ffmpeg vidókódoló, git, statikai számítás, stb.),
- külön python script.py-ket akarsz indítani és azok kimenetét feldolgozni.

**multiprocessing** viszont sokkal kényelmesebb, amikor:
- ugyanazt a Python-kódot akarod párhuzamosítani,
- függvényeket akarsz sok bemeneten futtatni,
- nem akarsz kézzel folyamatkezeléssel/kimenet-olvasással foglalkozni.

## Párhuzamos feldolgozás egyéb lehetőségei

Itt most csak a legegyszerűbb lehetőséget említettük, ami azokhoz a problémákhoz amikkel a leggyakrabban találkozunk elegendő. Párhuzamos feladatmegoldásra (céltól függően) más lehetőségek is vannak.
A felhaszálói felületek programozásánál gyakran alkalmazzák az Async megoldásokat (amely egyetlen processzormagon / szálon) is párhuzamos "hatást kelt" és interaktívabb felhasználói élményt nyújt, a hatékonyabb párhuzamos feldolgozásnál pedig szálakat (threads), amely kevésbé pazarló és közös memóriahasználatot tesz lehetővé.

Természetesen a pythonban ezekhez is van (több) csomag (is), annak ellenére, hogy nem foglalkoztunk részletesen velük. A python saját belső ilyen csomagjai:

* Asyncio: https://docs.python.org/3/library/asyncio.html
* Threading: https://docs.python.org/3/library/threading.html